In [2]:
import tensorflow as tf
print(tf.__version__)

2.2.0-rc4


##Load data

In [3]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteOQCE8P/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteOQCE8P/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteOQCE8P/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [0]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


##Data process

In [0]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)



In [6]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print("")
print(training_sentences[1])

? ? ? ? ? ? ? b'i have been known to fall asleep during films but this is usually due to a combination of things including really tired being warm and comfortable on the <OOV> and having just eaten a lot however on this occasion i fell asleep because the film was rubbish the plot development was constant constantly slow and boring things seemed to happen but with no explanation of what was causing them or why i admit i may have missed part of the film but i watched the majority of it and everything just seemed to happen of its own <OOV> without any real concern for anything else i cant recommend this film at all '

b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of

##Make the model

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


## Fit and save the whole model

In [8]:
num_epochs = 10
history = model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final), verbose=2)

Epoch 1/10
782/782 - 5s - loss: 0.5019 - accuracy: 0.7336 - val_loss: 0.3461 - val_accuracy: 0.8491
Epoch 2/10
782/782 - 5s - loss: 0.2496 - accuracy: 0.9022 - val_loss: 0.3588 - val_accuracy: 0.8439
Epoch 3/10
782/782 - 5s - loss: 0.1044 - accuracy: 0.9722 - val_loss: 0.4421 - val_accuracy: 0.8292
Epoch 4/10
782/782 - 5s - loss: 0.0295 - accuracy: 0.9954 - val_loss: 0.5259 - val_accuracy: 0.8279
Epoch 5/10
782/782 - 5s - loss: 0.0073 - accuracy: 0.9995 - val_loss: 0.5806 - val_accuracy: 0.8292
Epoch 6/10
782/782 - 5s - loss: 0.0024 - accuracy: 0.9999 - val_loss: 0.6365 - val_accuracy: 0.8283
Epoch 7/10
782/782 - 5s - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.6776 - val_accuracy: 0.8300
Epoch 8/10
782/782 - 5s - loss: 5.0985e-04 - accuracy: 1.0000 - val_loss: 0.7195 - val_accuracy: 0.8300
Epoch 9/10
782/782 - 5s - loss: 2.9290e-04 - accuracy: 1.0000 - val_loss: 0.7564 - val_accuracy: 0.8310
Epoch 10/10
782/782 - 5s - loss: 1.7378e-04 - accuracy: 1.0000 - val_loss: 0.7902 - val_accu

In [12]:
# Re-evaluate the model
loss,acc = model.evaluate(testing_padded, testing_labels_final, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

782/782 - 2s - loss: 0.7902 - accuracy: 0.8308
Restored model, accuracy: 83.08%


####Save the model by SavedModel format

In [17]:
# Save the entire model as a SavedModel.
model.save('saved_model/my_model') 

INFO:tensorflow:Assets written to: saved_model/my_model/assets


INFO:tensorflow:Assets written to: saved_model/my_model/assets


####Save the model by HDF5 format

In [0]:
%mkdir saved_model_h5
model.save('saved_model_h5/my_model.h5') 

##Load the saved model

####Load the model by SavedModel format

In [13]:
new_model = tf.keras.models.load_model('saved_model/my_model')
# Check its architecture
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Re-evaluate the model
loss,acc = new_model.evaluate(testing_padded, testing_labels_final, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

782/782 - 2s - loss: 0.7902 - accuracy: 0.8308
Restored model, accuracy: 83.08%


####Load the model by HDF5 format

In [20]:
new_model_h5 = tf.keras.models.load_model('saved_model_h5/my_model.h5')
# Check its architecture
new_model_h5.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Re-evaluate the model
loss,acc = new_model_h5.evaluate(testing_padded, testing_labels_final, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

782/782 - 2s - loss: 0.7902 - accuracy: 0.8308
Restored model, accuracy: 83.08%


Reference: https://www.tensorflow.org/tutorials/keras/save_and_load#save_the_entire_model